# Stored Procedure

In [1]:
# Function to install and import packages
def install_and_import(package):
    import importlib
    try:
        importlib.import_module(package)
    except ImportError:
        import pip
        pip.main(['install', package])
    finally:
        globals()[package] = importlib.import_module(package)

In [2]:
# Packages needed
install_and_import("pyodbc")
install_and_import("pandas")

In [3]:
# Database server connection
conn = pyodbc.connect(
       DRIVER='{ODBC Driver 17 for SQL Server}',  # Check driver version in ODBC Data Source Administrator in your personal computer
       SERVER = 'DESKTOP-CC1UAQG\SQL2019',        # SQL server name that can be seen when lauch SQL server 
       DATABASE = 'Survey_Sample_A19', 
       trusted_Connection = 'Yes')

# Create cursor to traverse over the records in a database
cur = conn.cursor()

In [4]:
# Create class for stored procedure

class StoredProcedure:
     
    def __init__(self):
        print("Instance of Stored Procedure called")
    
       
    # Function to get survey structure
    def getSurveyStructure(self, nameCSVinString, dbconn):
        query_SurveyStructure = pandas.read_sql_query("SELECT * FROM SurveyStructure ORDER BY SurveyId", dbconn)
        df_SurveyStructure = pandas.DataFrame(query_SurveyStructure)
        # Save original survey structure table in local drive
        df_SurveyStructure.to_csv(nameCSVinString, index=False)
        return df_SurveyStructure
    
    
    # Get all surveyId 
    def allSurveyId (self):
        query_ColSurveyId = "SELECT DISTINCT SurveyId FROM Answer ORDER BY SurveyId"
        cur.execute(query_ColSurveyId)
        row = [item[0] for item in cur.fetchall()]
        return row

    
    # Get all question Id
    def allQuestionId (self):
        query_ColQuestionId = "SELECT DISTINCT QuestionId FROM Answer ORDER BY QuestionId"
        cur.execute(query_ColQuestionId)
        row = [item[0] for item in cur.fetchall()]
        return row
    
    
    # Function to verify the question in each survey
    def AllSurvey_Question(self,SurveyList):
        checkQuestion_List = [ ]

        for surveyId in SurveyList:
            if surveyId < len(SurveyList):
                query_CheckQ_inSurvey = "SELECT * FROM (SELECT SurveyId, QuestionId, 1 as InSurvey FROM SurveyStructure WHERE " \
                                       "SurveyId = " +str(surveyId)+ \
                                       " UNION "  \
                                       "SELECT " +str(surveyId)+ " as SurveyId, Q.QuestionId, 0 as InSurvey" \
                                       " FROM Question as Q WHERE NOT EXISTS" \
                                       "(SELECT * FROM SurveyStructure as S WHERE S.SurveyId = " +str(surveyId)+\
                                       " AND S.QuestionId = Q.QuestionId)) as t UNION "
                checkQuestion = query_CheckQ_inSurvey 
                checkQuestion_List.append(checkQuestion)

            else: 
                query_CheckQ_LastSurvey = "SELECT * FROM (SELECT SurveyId, QuestionId, 1 as InSurvey FROM SurveyStructure WHERE " \
                                       "SurveyId = " +str(surveyId)+ \
                                       " UNION "  \
                                       "SELECT " +str(surveyId)+ " as SurveyId, Q.QuestionId, 0 as InSurvey" \
                                       " FROM Question as Q WHERE NOT EXISTS" \
                                       "(SELECT * FROM SurveyStructure as S WHERE S.SurveyId = " +str(surveyId)+\
                                       " AND S.QuestionId = Q.QuestionId)) as t ORDER BY SurveyId " 
                checkQuestion = query_CheckQ_LastSurvey
                checkQuestion_List.append(checkQuestion)


            allsurvey_question = " ".join(checkQuestion_List)
            surveyId  += 1
        return allsurvey_question
    
    
    # Create data frame to see all question in each query
    def getAllQuestioninSurvey(self, query, dbconn):
        query_allQuestioninSurvey = pandas.read_sql_query(query, dbconn)
        df_allQuestioninSurvey = pandas.DataFrame(query_allQuestioninSurvey)
        return df_allQuestioninSurvey

    
    # Make list of question in each survey 
    def getAllInsurveyQuestion(self,df):
        df_InSurvey = df.loc[df["InSurvey"]==1]
        InSurvey_list = df_InSurvey.values[:,[0,1]].tolist()
        return InSurvey_list


  # Select only user who gives response
    def Final(self,SurveyList, QuestionList, List):
        strCurrentUnionQueryBlock_List = [ ]
        strFinalQuery = " "

        # Match answer to question
        def DynamicAnswer(surveyId,questionId,List):
            strColumnsQueryPart = " "
            AnswerCol = "COALESCE((SELECT a.Answer_Value FROM Answer as a "\
                        "WHERE a.UserId = u.UserId AND a.SurveyId = " +str(surveyId)+\
                        " AND a.QuestionId = "+str(questionId)+"),-1) "\
                        "AS ANS_Q" +str(questionId)
            NullCol = " NULL AS ANS_Q" +str(questionId) 
            value = [surveyId,questionId]
            
            if value in List:
                strColumnsQueryPart = AnswerCol
            else:
                strColumnsQueryPart = NullCol
            return strColumnsQueryPart
    
       

        for surveyId in SurveyList:
            strQueryTemplateOuterUnionQuery = " SELECT UserId, " +str(surveyId)+ " as SurveyId, "

            for questionId in QuestionList:
                if questionId in range(0,len(QuestionList)):
                    strQueryTemplateOuterUnionQuery = strQueryTemplateOuterUnionQuery + DynamicAnswer(surveyId,questionId,List) + ","
                else: 
                    strQueryTemplateOuterUnionQuery = strQueryTemplateOuterUnionQuery + DynamicAnswer(surveyId,questionId,List) + " FROM [User] as u" \
                                                      " WHERE EXISTS (SELECT * FROM Answer as a WHERE u.UserId = a.UserId" \
                                                      " AND a.SurveyId = " +str(surveyId)+ ") UNION "
                    strCurrentUnionQueryBlock_List.append(strQueryTemplateOuterUnionQuery)
                    strCurrentUnionQueryBlock = " ".join(strCurrentUnionQueryBlock_List)
                questionId += 1
            surveyId += 1
            strFinalQuery = " ".join(strCurrentUnionQueryBlock.split(" ")[:-2])
        return strFinalQuery


    # Get final survey data
    def Output(self, query,dbconn, nameCSVinString):
        query_Output = pandas.read_sql_query(query, dbconn)
        df_Output = pandas.DataFrame(query_Output)
        df_Output.to_csv(nameCSVinString, index=False)
        return df_Output.head(5)   # Check output
    
    
    # Create view
    def createView(self, View_NameInString, query, dbcur, dbconn):
        query_view = "CREATE VIEW " + View_NameInString + " AS " + query
        dbcur.execute(query_view)
        dbconn.commit()
        
        
    # Close connection
    def dbClose(self, dbconn, dbcur):
        dbcur.close()
        del dbcur
        dbconn.close()

In [5]:
# Create class attribute
db_A19 = StoredProcedure()

Instance of Stored Procedure called


In [6]:
# Get original survey structure table to compare the change of database later
df_SurveyStructure = db_A19.getSurveyStructure("SurveyStructure_Khwansiri.csv", conn)
df_SurveyStructure

,SurveyId,QuestionId,OrdinalValue
0,1,1,1
1,1,2,2
2,2,2,1
3,2,3,2


In [7]:
# Get all surveyId 
allSurveyId_list = db_A19.allSurveyId()
allSurveyId_list

[1, 2, 3]

In [8]:
# Get all question Id
allQuestionId_list = db_A19.allQuestionId()
allQuestionId_list

[1, 2, 3]

In [9]:
# Function to verify the question in each survey
QuestionInSurvey = db_A19.AllSurvey_Question(allSurveyId_list)

In [10]:
# Create data frame to see all question in each query
df_AllQuestion =  db_A19.getAllQuestioninSurvey(QuestionInSurvey, conn)
df_AllQuestion

,SurveyId,QuestionId,InSurvey
0,1,1,1
1,1,2,1
2,1,3,0
3,1,4,0
4,2,1,0
5,2,2,1
6,2,3,1
7,2,4,0
8,3,1,0
9,3,2,0


In [11]:
# Make list of question in each survey 
InSurvey_list =  db_A19.getAllInsurveyQuestion(df_AllQuestion)
InSurvey_list

[[1, 1], [1, 2], [2, 2], [2, 3]]

In [12]:
# Match answer to question
FinalQuery = db_A19.Final(allQuestionId_list, allSurveyId_list, InSurvey_list)
FinalQuery

' SELECT UserId, 1 as SurveyId, COALESCE((SELECT a.Answer_Value FROM Answer as a WHERE a.UserId = u.UserId AND a.SurveyId = 1 AND a.QuestionId = 1),-1) AS ANS_Q1,COALESCE((SELECT a.Answer_Value FROM Answer as a WHERE a.UserId = u.UserId AND a.SurveyId = 1 AND a.QuestionId = 2),-1) AS ANS_Q2, NULL AS ANS_Q3 FROM [User] as u WHERE EXISTS (SELECT * FROM Answer as a WHERE u.UserId = a.UserId AND a.SurveyId = 1) UNION   SELECT UserId, 2 as SurveyId,  NULL AS ANS_Q1,COALESCE((SELECT a.Answer_Value FROM Answer as a WHERE a.UserId = u.UserId AND a.SurveyId = 2 AND a.QuestionId = 2),-1) AS ANS_Q2,COALESCE((SELECT a.Answer_Value FROM Answer as a WHERE a.UserId = u.UserId AND a.SurveyId = 2 AND a.QuestionId = 3),-1) AS ANS_Q3 FROM [User] as u WHERE EXISTS (SELECT * FROM Answer as a WHERE u.UserId = a.UserId AND a.SurveyId = 2) UNION   SELECT UserId, 3 as SurveyId,  NULL AS ANS_Q1, NULL AS ANS_Q2, NULL AS ANS_Q3 FROM [User] as u WHERE EXISTS (SELECT * FROM Answer as a WHERE u.UserId = a.UserId AND

In [13]:
# Get final survey data
AllSurveyData = db_A19.Output(FinalQuery, conn, "GetAllSurveyDataSQL_Khwansiri.csv")
AllSurveyData

,UserId,SurveyId,ANS_Q1,ANS_Q2,ANS_Q3
0,42,1,3.0,-1.0,NaN
1,296,1,5.0,-1.0,NaN
2,1793,1,-1.0,6.0,NaN
3,2411,1,-1.0,1.0,NaN
4,2634,1,3.0,-1.0,NaN


In [14]:
# Create view
db_A19.createView("vw_AllSurveyData_KN", FinalQuery, cur, conn)

In [15]:
# Close connection
db_A19.dbClose(conn,cur)